In [1]:
from keybert import KeyBERT
from transformers import pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import json
import mysql.connector
from sqlalchemy import create_engine
from tqdm import tqdm 

In [2]:
#dataset = pd.read_csv('gxs_reviews.csv', index_col=0)

In [4]:
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Establish a connection using the loaded configuration
cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM gxs_apple_app_reviews"

# Use pandas to load sql query into a DataFrame
gxsapple = pd.read_sql(query, con=cnx)

# Don't forget to close the connection when done
cnx.close()

C:\Users\AngJi\AppData\Local\Temp\ipykernel_16860\3694027278.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gxsapple = pd.read_sql(query, con=cnx)


In [8]:
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Establish a connection using the loaded configuration
cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM gxs_playstore_app_reviews"

# Use pandas to load sql query into a DataFrame
gxsplaystore = pd.read_sql(query, con=cnx)

# Don't forget to close the connection when done
cnx.close()

C:\Users\AngJi\AppData\Local\Temp\ipykernel_16860\1891013442.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gxsplaystore = pd.read_sql(query, con=cnx)


In [5]:
gxsapple['title_review'] = gxsapple['title'] + ' : ' + gxsapple['review']

gxsapple['thumbsUp']= 0

gxsapple['developerResponse'] = gxsapple['developerResponse'].apply(lambda x: x.get('body') if isinstance(x, dict) else x)
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace(r"\{'id': \d+, 'body':", "", regex=True)
gxsapple['developerResponse']=gxsapple['developerResponse'].str.rstrip("}")
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace('"', '', regex=False)
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace("'", '', regex=False)

gxsapple_dropped = gxsapple.drop(columns=['title', 'review', 'userName', 'isEdited'])

gxsplaystore_dropped=gxsplaystore.drop(columns=['reviewId','userName','userImage','reviewCreatedVersion','repliedAt','appVersion'])
neworder=['replyContent', 'score', 'content', 'thumbsUpCount', 'at']
gxsplaystore_dropped=gxsplaystore_dropped[neworder]

gxsplaystore_dropped=gxsplaystore_dropped.rename(columns={
    'content': 'content',
    'thumbsUpCount': 'thumbsUpCount',
    'replyContent': 'replyContent',
    'score': 'score',
    'at':'date'
})

gxsapple_dropped_renamed = gxsapple_dropped.rename(columns={
    'title_review': 'content',
    'thumbsUp': 'thumbsUpCount',
    'developerResponse': 'replyContent',
    'rating': 'score',
    'date':'date'
})

dataset = pd.concat([gxsplaystore_dropped, gxsapple_dropped_renamed], axis=0, ignore_index=True)
if dataset['replyContent'].apply(lambda x: isinstance(x, dict)).any():
    dataset['replyContent'] = dataset['replyContent'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)


In [6]:
dataset.head()

,replyContent,score,content,thumbsUpCount,date
0,Hi there! Thank you for your 5-star rating rev...,5,I like how the app is really light and fast co...,0,2024-04-15 02:17:49
1,"Hi Jerry, this feature isn't ready at the mome...",1,Cannot add card to google pay. Also cannot add...,0,2024-04-13 08:22:18
2,Thanks for your feedback on mobile wallets and...,4,Generally good. However please add either NFC ...,0,2024-04-13 05:53:34
3,Hi Eric. We're sorry to hear this. We would li...,1,"With continuous failed to log in,contacted cso...",0,2024-04-11 16:59:52
4,Hi Hafidz Melek! Thank you for your 5-star rat...,5,I don't get the negative reviews here. I insta...,0,2024-04-09 13:58:31


In [16]:
sa_model = 'ajiayi/finetuning-sentiment-model-bank_reviews-otherbank'
sentiment_pipeline = pipeline("sentiment-analysis", model=sa_model)

In [4]:
ke_model = KeyBERT()

In [ ]:
baseline_model = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(baseline_model)

save_path = "./gpt_model_causallm"
finetuned_model = AutoModelForCausalLM.from_pretrained(save_path)

generator = pipeline('text-generation', finetuned_model, tokenizer=tokenizer)
inst = 'Below is a review for a banking app. Write a response that appropriately replies to the review from the perspective of the bank.'

In [57]:
# sentiment = []
keywords = []
# replies = []

for index, row in dataset.iterrows():
    review = row['content']
    # s = sentiment_pipeline(review)[0]['label']
    # sentiment.append(s)
    k = ke_model.extract_keywords(review, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=1)
    if k == []:
        k = ''
    else: k = k[0][0]
    keywords.append(k)
    # prompt = f'{inst} ### Review: {review} ### Reply:'
    # reply = generator(prompt, max_length=len(review)+50)[0]['generated_text'][152+len(review):]
    # replies.append(reply)

# dataset['sentiment'] = sentiment
dataset['keywords'] = keywords
# dataset['generatedReply'] = replies

In [ ]:
# dataset = dataset.rename(columns={'replies':'generatedReply'})

In [58]:
dataset.head()

,replyContent,score,content,thumbsUpCount,date,keywords,sentiment
0,"Hey Jiaxing! Hope you love the new ""Favourites...",5,"Gxs is simple and easy to use, with a saving a...",13,2024-03-22 09:33:40,gxs simple,POSITIVE
1,We apologize for the inconvenience you had to ...,3,It always ask for finger identification even t...,0,2024-03-16 20:03:04,finger identification,NEGATIVE
2,"Hi Sean, we understand your concern and we wou...",1,What I've installed on my phone is none of you...,0,2024-03-16 15:18:24,using apps,NEGATIVE
3,"Hi Xiangqing, we'd love to look into this for ...",1,cant logging using finger print . previously s...,0,2024-03-10 23:51:02,finger print,NEGATIVE
4,"Hi Matt, thanks for reaching out! If you are r...",1,For 1st time registration signup by singpass a...,0,2024-03-10 22:02:20,signup singpass,NEGATIVE


In [59]:
dataset.keywords

0                 gxs simple
1      finger identification
2                 using apps
3               finger print
4            signup singpass
               ...          
326                  wow wow
327                         
328                 scan pay
329                 bug 安全漏洞
330              singpass qr
Name: keywords, Length: 331, dtype: object

In [62]:
dataset2 = pd.read_csv('sentiment_data.csv', index_col=0)

In [64]:
# dataset2['keywords'] = keywords

In [65]:
# dataset2

,replyContent,score,content,thumbsUpCount,date,sentiment,keywords,generatedReply
0,"Hey Jiaxing! Hope you love the new ""Favourites...",5,"Gxs is simple and easy to use, with a saving a...",13,2024-03-22 09:33:40,POSITIVE,gxs simple,Hi Matt! Welcome back to the review from the ...
1,We apologize for the inconvenience you had to ...,3,It always ask for finger identification even t...,0,2024-03-16 20:03:04,NEGATIVE,finger identification,Great! I appreciate that your prompt attentio...
2,"Hi Sean, we understand your concern and we wou...",1,What I've installed on my phone is none of you...,0,2024-03-16 15:18:24,NEGATIVE,using apps,Please help us understand if you see any of t...
3,"Hi Xiangqing, we'd love to look into this for ...",1,cant logging using finger print . previously s...,0,2024-03-10 23:51:02,NEGATIVE,finger print,uhhh the problem is i already have rooted the...
4,"Hi Matt, thanks for reaching out! If you are r...",1,For 1st time registration signup by singpass a...,0,2024-03-10 22:02:20,NEGATIVE,signup singpass,"Hi L.L., We are sorry for this inconvenience...."
...,...,...,...,...,...,...,...,...
326,Thank you very much for your kind support! We...,5,Wow : Wow,0,2023-09-29 13:18:27,POSITIVE,wow wow,"Hello again, Thank you for your feedbacks! 🙏 ..."
327,Hey Jasper! We are happy to know that you lik...,5,🚀🚀🚀🚀🚀 : 🚀🚀🚀🚀🚀,0,2022-09-16 18:40:11,POSITIVE,,Hi
328,Hi there. We apologize for the inconvenience ...,1,Scan and pay amount wrong : I was trying to ma...,0,2023-07-24 03:31:10,NEGATIVE,scan pay,"Hi, we are sorry to see your situation. We ar..."
329,Hi there! Were sorry you feel that way. Fret ...,1,Bug ! Bug ! Bug ! : 安全漏洞！下载要小心！即使用SingPass 开户也...,0,2023-07-22 01:33:39,NEGATIVE,bug 安全漏洞,Sorry for the inconvenience


In [66]:
# dataset2.to_csv('sentiment_data.csv')

In [67]:
gxs_reviews = dataset[['content', 'thumbsUpCount', 'sentiment', 'keywords']]

In [77]:
def summarise_sentiment(reviews): #havent test this fn

  '''
  Input:
  - reviews: dataset containing at least the following 4 columns ['content', 'thumbsUpCount', 'sentimment', 'keywords']
  Output:
  - net sentiment score: numeric, between -100 and +100
  - neg_keywords: dictionary containing negative keywords, sorted descending
  - pos_keywords: dictionary containing positive keywords, sorted descending
  '''

  ns_score = 0
  neg_keywords = {}
  pos_keywords = {}

  for index, row in reviews.iterrows():
    pred_sentiment = row['sentiment']
    # keywords = row['keywords']
    word = row['keywords']
    if pred_sentiment == 'POSITIVE':
      pred_sentiment = 1
      # for word in keywords:
      if word in pos_keywords:
        pos_keywords[word] += 1
      else:
        pos_keywords[word] = 1
    else:
      pred_sentiment = -1
      # for word in keywords:
      if word in neg_keywords:
        neg_keywords[word] += 1
      else:
        neg_keywords[word] = 1
    ns_score += pred_sentiment

  total = len(reviews)
  ns_score = ns_score/total*100

  if '' in pos_keywords: del pos_keywords['']
  if '' in neg_keywords: del neg_keywords['']
  pos_keywords = dict(sorted(pos_keywords.items(), key=lambda x: x[1], reverse = True))
  neg_keywords = dict(sorted(neg_keywords.items(), key=lambda x: x[1], reverse = True))

  return ns_score, pos_keywords, neg_keywords

In [78]:
summary = summarise_sentiment(gxs_reviews)

In [79]:
summary

(-17.82477341389728,
 {'easy use': 5,
  'awesome app': 4,
  'good': 3,
  'ease use': 3,
  'banking app': 3,
  'great app': 3,
  'good app': 2,
  'gxs bank': 2,
  'uiux app': 2,
  'gxs': 2,
  'saving pocket': 2,
  'digital banking': 2,
  'bank app': 2,
  'app design': 2,
  'gxs simple': 1,
  'multiple pockets': 1,
  'fast repayments': 1,
  'ok good': 1,
  'using bank': 1,
  'savings pockets': 1,
  'load apps': 1,
  'fast app': 1,
  'easy loan': 1,
  'digital savings': 1,
  'great savings': 1,
  'loving far': 1,
  'rebates app': 1,
  'activate gxs': 1,
  'secure transactions': 1,
  'use payments': 1,
  'great bank': 1,
  'super efficient': 1,
  'superb': 1,
  'transfer bank': 1,
  'friendly user': 1,
  'loan foreigner': 1,
  'month gxs': 1,
  'easy fingertips': 1,
  'loan loan': 1,
  'cool app': 1,
  'seamless': 1,
  'love gxs': 1,
  'setup pocket': 1,
  'payee feature': 1,
  'apps easy': 1,
  'seamless registration': 1,
  'nice': 1,
  'app easy': 1,
  'easy register': 1,
  'sleek simple

In [80]:
summary_df = pd.DataFrame(summary, index=['netSentiment', 'positiveKeywords', 'negativeKeywords'])

In [81]:
# summary_df

,0
netSentiment,-17.824773
positiveKeywords,"{'easy use': 5, 'awesome app': 4, 'good': 3, '..."
negativeKeywords,"{'gxs bank': 4, 'gxs app': 3, 'account gxs': 2..."


In [83]:
summary_df.to_csv('summary.csv')

In [82]:
print(summary[1])

{'easy use': 5, 'awesome app': 4, 'good': 3, 'ease use': 3, 'banking app': 3, 'great app': 3, 'good app': 2, 'gxs bank': 2, 'uiux app': 2, 'gxs': 2, 'saving pocket': 2, 'digital banking': 2, 'bank app': 2, 'app design': 2, 'gxs simple': 1, 'multiple pockets': 1, 'fast repayments': 1, 'ok good': 1, 'using bank': 1, 'savings pockets': 1, 'load apps': 1, 'fast app': 1, 'easy loan': 1, 'digital savings': 1, 'great savings': 1, 'loving far': 1, 'rebates app': 1, 'activate gxs': 1, 'secure transactions': 1, 'use payments': 1, 'great bank': 1, 'super efficient': 1, 'superb': 1, 'transfer bank': 1, 'friendly user': 1, 'loan foreigner': 1, 'month gxs': 1, 'easy fingertips': 1, 'loan loan': 1, 'cool app': 1, 'seamless': 1, 'love gxs': 1, 'setup pocket': 1, 'payee feature': 1, 'apps easy': 1, 'seamless registration': 1, 'nice': 1, 'app easy': 1, 'easy register': 1, 'sleek simple': 1, 'flexiloan promotion': 1, 'bank amazing': 1, 'digibank mobile': 1, 'best bank': 1, 'interface app': 1, 'onboarding

In [45]:
print(summary[2])

{'gxs bank': 4, 'gxs app': 3, 'account gxs': 2, 'singpass qr': 2, 'open': 2, 'unable login': 2, 'buggy app': 2, 'app suddenly': 2, 'debit card': 2, 'banking app': 2, 'unable sign': 2, 'lousy app': 2, 'flexi loan': 2, 'finger identification': 1, 'using apps': 1, 'finger print': 1, 'signup singpass': 1, 'application 4x': 1, 'fingerprint login': 1, 'recipients fast': 1, 'foreign apply': 1, 'uninstalling trust': 1, 'gxbank worldwide': 1, 'months pass': 1, 'login': 1, 'stuck singpass': 1, 'sig': 1, 'money stuck': 1, 'malware hp': 1, 'open gxs': 1, 'phone modified': 1, 'terrible app': 1, 'favourite payee': 1, 'scam': 1, 'rating app': 1, 'sing pass': 1, 'email 2fa': 1, 'early access': 1, 'use bank': 1, 'worst rate': 1, 'false advertising': 1, 'closed account': 1, 'ssucks': 1, 'uninstalling': 1, 'waiting list': 1, 'paynow qr': 1, 'gxs flexiloan': 1, '2023 banking': 1, 'register pop': 1, 'login issue': 1, 'email late': 1, 'wrong apps': 1, 'open app': 1, 'email otp': 1, 'problem login': 1, 'fund